In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, sys
sys.path.append('/content/drive/MyDrive/CPD_BT')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import random

import itertools

from bt_cpd import *

import time
import bisect

import pandas as pd

import statsmodels.api as sm
from sklearn import linear_model

from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
T = 4
Delta = 2000
m = np.array([Delta] * T)
cp_truth = np.cumsum(m)[:T-1]

n = 100
beta = np.zeros((T, n))

t = 0.9
kappa = np.log(t / (1 - t))
delta = 1
beta_ref = get_beta_with_gap(n, delta)
beta_ref *= kappa / (np.max(beta_ref) - np.min(beta_ref))
beta[0] = beta_ref[:]
beta[1] = change_type(beta_ref, 1)
beta[2] = change_type(beta_ref, 2)
beta[3] = change_type(beta_ref, 3)

print(max(beta[0]) - min(beta[0]))

diff = np.zeros(T - 1)
for t in range(1, T):
    diff[t - 1] = np.sum(np.abs(beta[t] - beta[t - 1])**2)**0.5
print(diff)

2.1972245773362196
[12.81317951 11.09709382 12.81317951]


In [5]:
cp_truth

array([2000, 4000, 6000])

In [6]:
path = '/content/drive/MyDrive/CPD_BT/'
with open(path + 'data_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '.npy', 'rb') as f:
    X_train_list = np.load(f)
    Y_train_list = np.load(f)
    X_test_list = np.load(f)
    Y_test_list = np.load(f)

In [7]:
X_train_list.shape

(100, 8000, 100)

In [8]:
np.random.seed(0)

m_intervals = 50
grid_n = 250
gamma_list = [80, 100]
lam_list = [0.1]

nt = Delta * T
B = 100

run_time_wbs = np.zeros(B)
loc_error_wbs = np.zeros(B)
K_wbs = np.zeros(B)

cp_best_list = []
param_best_list = []

for b in range(B):
    X_train = X_train_list[b]
    Y_train = Y_train_list[b]
    X_test = X_test_list[b]
    Y_test = Y_test_list[b]

    start_time = time.time()
    wbs_fit = wbs_cv_bt(m_intervals, grid_n, lam_list, gamma_list, smooth = 5, buffer = 5)
    res = wbs_fit.fit((X_train, Y_train), (X_test, Y_test))
    cp_best, cp_val, cusum_val, threshold_best, grid = res   
    run_time_wbs[b] = time.time() - start_time
    loc_error_wbs[b] = cp_distance(cp_best, cp_truth)
    K_wbs[b] = len(cp_best)

    cp_best_list.append(cp_best)
    param_best_list.append(threshold_best)

    print(b)

print('---------- wbs -----------')
print("avg loc error: {0}, avg time: {1}".format(loc_error_wbs.mean(), run_time_wbs.mean()))
print("std loc error: {0}, std time: {1}".format(loc_error_wbs.std(), run_time_wbs.std()))
print('K < K*: {0}, K = K*: {1}, K > K*: {2}'.format(sum(K_wbs < T - 1), sum(K_wbs == T - 1), sum(K_wbs > T - 1)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
---------- wbs -----------
avg loc error: 412.32, avg time: 399.90618260145186
std loc error: 495.53968317381, std time: 40.900529064217125
K < K*: 0, K = K*: 57, K > K*: 43


In [9]:
loc_error_wbs

array([ 528.,   16.,   16.,  784.,   16.,  560.,   16.,   16.,   16.,
         16., 1040.,   32.,  944.,   16.,   16.,   16.,   16.,   16.,
       1040., 1232., 1264.,   16.,   16., 1488.,  752.,   32.,   16.,
         16., 1264.,   16.,   16.,   16.,   16., 1264.,  400.,  800.,
        720.,  704.,   16., 1264.,  720.,  928.,  832.,   16., 1456.,
         16.,   16.,  640.,   16.,   16.,  816.,   16., 1072.,   16.,
        944.,   16.,   16.,  496.,   16.,   16.,  976.,   16.,   16.,
        896.,   16.,   48.,  720.,   16.,   16.,   16., 1136.,   16.,
         16.,   16., 1488., 1616.,   16.,   16., 1104.,  768.,  608.,
       1392.,  864.,   16.,  800.,  592.,   16.,  944.,   16.,  768.,
         16.,   16., 1104.,   16.,   16.,   16.,   16.,  528.,   16.,
         16.])

In [10]:
param_best_list

[80,
 100,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 100,
 80,
 80,
 80,
 100,
 80,
 100,
 100,
 100,
 80,
 80,
 80,
 80,
 100,
 80,
 80,
 100,
 100,
 100,
 80,
 100,
 100,
 80,
 100,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 80,
 100,
 80,
 80,
 100,
 80,
 100,
 100,
 80,
 80,
 80,
 100,
 80,
 80,
 80,
 80,
 100,
 80,
 80,
 80,
 100,
 80,
 100,
 100,
 80,
 80,
 100,
 100,
 80,
 100,
 100,
 80,
 80,
 80,
 100,
 80,
 80,
 80,
 80,
 80,
 80,
 100,
 80,
 80,
 100,
 80,
 80,
 80,
 80,
 100,
 80,
 100,
 100,
 100,
 100,
 80,
 100,
 100]

In [11]:
cp_best_list

[[1984, 4000, 5984, 6528],
 [1984, 4000, 5984],
 [1984, 4000, 5984],
 [2016, 2560, 4000, 6016, 6784],
 [1984, 4000, 5984],
 [1984, 2560, 4000, 6016],
 [1984, 4000, 6016],
 [2016, 4000, 5984],
 [1984, 4000, 6016],
 [2016, 4000, 5984],
 [1984, 4000, 5216, 5984, 7040],
 [1984, 3968, 6016],
 [1984, 4000, 5056, 5984],
 [1984, 4000, 6016],
 [1984, 4000, 5984],
 [1984, 4000, 5984],
 [2016, 4000, 5984],
 [1984, 4000, 5984],
 [2016, 4000, 5984, 6688, 7040],
 [768, 1504, 1984, 4000, 5152, 5984],
 [736, 2016, 3968, 5984],
 [2016, 4000, 5984],
 [2016, 4000, 5984],
 [1536, 2016, 4000, 5984, 7488],
 [2016, 4000, 5344, 6016, 6752],
 [1984, 3968, 5984],
 [1984, 4000, 5984],
 [2016, 4000, 6016],
 [736, 1984, 4000, 4416, 6016],
 [1984, 4000, 6016],
 [2016, 4000, 5984],
 [1984, 4000, 6016],
 [2016, 4000, 5984],
 [736, 1984, 4000, 5984, 6464],
 [1632, 1984, 4000, 5600, 6016],
 [1984, 3200, 4000, 5344, 5984],
 [2016, 4000, 6016, 6720],
 [2016, 3296, 4000, 5984],
 [1984, 4000, 6016],
 [1984, 4000, 5440, 601

In [12]:
import pickle
with open(path + 'wbs_n' + str(n) + '_Delta' + str(Delta) + '_K' + str(T - 1) + '_grid' + str(grid_n) + '.pickle', 'wb') as f:
    pickle.dump([cp_best_list, param_best_list, loc_error_wbs, run_time_wbs, K_wbs], f)